<a href="https://colab.research.google.com/github/dashatenoff/recsys-vk/blob/main/notebooks/EASE_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EASE Recommender System (VK-LSVD)

"""
В этом ноутбуке реализована модель рекомендательной системы
на основе implicit feedback с использованием модели EASE.

EASE — линейная item-item модель, которая восстанавливает
матрицу взаимодействий пользователей и объектов через саму себя.

Цель работы — построить baseline-модель рекомендаций,
оценить её качество по метрике MAP@10
и сравнить с ALS-подходом.
"""


In [ ]:
import pandas as pd
from google.colab import drive
import os
import numpy as np
from scipy.sparse import csr_matrix
from tqdm import tqdm

drive.mount('/content/drive')
train = pd.read_parquet('/content/drive/MyDrive/VK/train.parquet')
test = pd.read_parquet('/content/drive/MyDrive/VK/test.parquet')

train['timespent'].describe()


#Подготовка данных (маппинг + CSR)

In [ ]:
user_to_index = {}
item_to_index = {}
rows, cols, data = [], [], []

for row in train.itertuples():
    u = user_to_index.setdefault(row.user_id, len(user_to_index))
    i = item_to_index.setdefault(row.item_id, len(item_to_index))

    rows.append(u)
    cols.append(i)
    data.append(1)

user_item_matrix = csr_matrix((data, (rows, cols)))


#Реализация EASE


In [ ]:
from scipy import sparse
def train_ease_model(
    interactions,
    regularization: float = 0.01,
):
  gram_matrix = interactions.T @ interactions
  gram_matrix += regularization * sparse.identity(gram_matrix.shape[0]).astype(np.float32)
  gram_matrix = gram_matrix.todense()
  gram_matrix_inv = np.linalg.inv(gram_matrix)
  weight = np.array(gram_matrix_inv / (-np.diag(gram_matrix_inv)))
  np.fill_diagonal(weight, 0.0)
  return weight

#Сигнатура функции

In [ ]:
from typing import Dict

def generate_ease_recommendations(
        interactions: np.ndarray,
        ease_matrix: np.ndarray,
        user_to_index: Dict,
        item_to_index: Dict,
        k: int = 10,
    ) -> pd.DataFrame:
    """
    Генерирует рекомендации с использованием обученной EASE модели
    """
    # Создаем обратные маппинги
    index_to_item = {v: k for k, v in item_to_index.items()}

    # Вычисляем предсказания
    predicted_scores = interactions @ ease_matrix

    recommendations = []

    for user_id, user_idx in tqdm(user_to_index.items(), desc="Генерация рекомендаций"):
        user_scores = predicted_scores[user_idx]

        seen_items = interactions[user_idx].nonzero()[0]
        user_scores[seen_items] = -np.inf
        # Сортируем айтемы по убыванию скоров
        item_indices = np.argsort(user_scores)[::-1]

        # Формируем рекомендации
        user_recs = []
        for item_idx in item_indices:
            if len(user_recs) >= k:
                break
            item_id = index_to_item[item_idx]
            user_recs.append(item_id)

        recommendations.append({
            "user_id": user_id,
            "recommendations": user_recs
        })

    return pd.DataFrame(recommendations)

#pipeline для EASE + MAP@10

In [ ]:
ease_matrix = train_ease_model(
    interactions=user_item_matrix,
    regularization=2000
)


ease_recs = generate_ease_recommendations(
    interactions=user_item_matrix.toarray(),
    ease_matrix=ease_matrix,
    user_to_index=user_to_index,
    item_to_index=item_to_index,
    k=10
)

recs = []

for _, row in ease_recs.iterrows():
    user_id = row['user_id']
    for item in row['recommendations']:
        recs.append({
            'user_id': user_id,
            'recs': item
        })

submission = pd.DataFrame(recs)

rel = test.groupby('user_id')['item_id'].apply(set)
pred = submission.groupby('user_id')['recs'].apply(list)

aps = []

for user_id in pred.index:
    relevant_items = rel.get(user_id, set())

    if len(relevant_items) == 0:
        continue

    ord = 1
    cor = 0
    score = 0.0

    for item_id in pred[user_id]:
        if item_id in relevant_items:
            cor += 1
            score += cor / ord
        ord += 1

    aps.append(score / min(len(relevant_items), 10))

map10 = sum(aps) / len(aps)
map10



## Результаты и интерпретация

Для модели **EASE** было получено значение:

**MAP@10 = 0.0087**

Данный результат сопоставим с простыми бейзлайн-моделями
и существенно ниже, чем у Implicit ALS.

Это объясняется следующими причинами:
- EASE является **линейной item-item моделью** и не использует
  латентные пользовательские представления;
- модель чувствительна к разреженности данных и шуму
  в implicit-взаимодействиях;
- отсутствует механизм усиления персонализации,
  характерный для факторизационных моделей.

В сравнении:
- **Implicit ALS** лучше захватывает скрытые предпочтения пользователей
  за счёт эмбеддингов;
- **EASE** выигрывает в простоте и скорости,
  но уступает по качеству рекомендаций на данном датасете.

Таким образом, EASE можно рассматривать как быстрый baseline,
в то время как ALS является более предпочтительным решением
для production-like сценариев.
